In [1]:
import scanpy as sc
import numpy as np 
import pandas as pd
# openpyxl package to read excel# 'pip install openpyxl' if undownload it

In [2]:
supplementaryTable = "Supplementary table.xlsx"
dataset = "PBMC68k"
dataset_url = "https://cblast.gao-lab.org/Zheng/Zheng.h5"
CellType_ColumnName = 'cell_ontology_class'

In [3]:
# dataset download
from os.path import isfile
dataset_download_path = dataset_url.split('/')[-1]
if not isfile(dataset_download_path):
    !wget -O {dataset_download_path} {dataset_url}

--2022-03-04 11:37:15--  https://cblast.gao-lab.org/Zheng/Zheng.h5
Resolving cblast.gao-lab.org (cblast.gao-lab.org)... 159.138.49.219
Connecting to cblast.gao-lab.org (cblast.gao-lab.org)|159.138.49.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119125320 (114M) [application/octet-stream]
Saving to: ‘Zheng.h5’

Zheng.h5            100%[===================>] 113.61M  6.44MB/s    in 19s     

2022-03-04 11:37:35 (6.07 MB/s) - ‘Zheng.h5’ saved [119125320/119125320]



In [18]:
def read_10x_h5(filename):
    import h5py
    import scanpy as sc
    from scipy import sparse
    import pandas as pd
    ds = h5py.File(filename, 'r')
    m=ds['exprs']
    def get_para(para):
        re = {}
        for k in ds[para].keys():
            re[k] = ds[para][k][:].astype(str)
        return re   
    ad=sc.AnnData(
      X = sparse.csr_matrix((m['data'][:],m['indices'][:],m['indptr'][:]),shape=m['shape']).tocsr(),
      var = pd.DataFrame(get_para('var'),index=ds['var_names'][:].astype(str)),
      obs = pd.DataFrame(get_para('obs'),index=ds['obs_names'][:].astype(str)),
      uns = get_para('uns')
    )
    return ad

In [5]:
adata = read_10x_h5(dataset_download_path)
adata.X = adata.X.tocsr()
df = pd.read_excel(supplementaryTable, sheet_name=dataset)

In [6]:
def add_celltype(adata,celltype_columnname):
    adata.obs[celltype_columnname] = adata.obs[celltype_columnname].astype("category")
    Transferlist = [np.where(df['origin cell type']==CellTypeList)[0][0] for CellTypeList in adata.obs[celltype_columnname].values.categories]
    adata.obs['modelA id'] = df['modelA id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['modelC id'] = df['modelC id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['transfer cell type'] = df['transfer cell type'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
add_celltype(adata,CellType_ColumnName)

In [ ]:
adata.obs['origin cell type'] = adata.obs['cell_ontology_class']

In [43]:
adata.obs

,cell_ontology_class,cell_ontology_id,cell_type1,dataset_name,latent_1,latent_10,latent_2,latent_3,latent_4,latent_5,...,latent_8,latent_9,organ,organism,platform,tSNE1,tSNE2,modelA id,modelC id,transfer cell type
1-AAACATACATTCTC-1,hematopoietic precursor cell,CL:0008001,CD34+,Zheng,3.5131507,2.2871785,-0.6718394,-0.22036982,0.48641413,2.1567016,...,-0.18197304,-0.8186109,PBMC,Homo sapiens,10x,1.5541508,-38.776894,12,10,bone marrow hematopoietic cell
1-AAACATTGCGATAC-1,hematopoietic precursor cell,CL:0008001,CD34+,Zheng,1.0836844,3.3510747,-0.6018521,-0.5259382,2.3784885,0.7962606,...,-0.20721523,-0.5377821,PBMC,Homo sapiens,10x,-6.2046695,-32.29672,12,10,bone marrow hematopoietic cell
1-AAACATTGCGTCTC-1,hematopoietic precursor cell,CL:0008001,CD34+,Zheng,1.9266928,1.6855874,1.22465,0.77250063,2.6336217,0.21636468,...,0.47245127,-1.2944603,PBMC,Homo sapiens,10x,-25.405148,-30.10589,12,10,bone marrow hematopoietic cell
1-AAACATTGGCCAAT-1,hematopoietic precursor cell,CL:0008001,CD34+,Zheng,2.452773,2.0287545,-0.16491967,0.3953306,1.5189058,1.215086,...,-0.90346986,-0.84549,PBMC,Homo sapiens,10x,-2.8238227,-35.632774,12,10,bone marrow hematopoietic cell
1-AAACATTGGCCATA-1,hematopoietic precursor cell,CL:0008001,CD34+,Zheng,2.6564171,1.0019425,0.11461637,-0.5024492,2.3206618,1.2925202,...,-0.028891545,-0.34730726,PBMC,Homo sapiens,10x,-9.481107,-36.98051,12,10,bone marrow hematopoietic cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10-TGTCAGGACTACTT-1,dendritic cell,CL:0000451,Dendritic,Zheng,0.7732218,5.4901876,2.1204052,-0.4125584,0.9447458,-3.4585948,...,-1.9096261,0.7374275,PBMC,Homo sapiens,10x,8.496808,-43.700546,11,9,dendritic cell
10-TTAGGGACCTGACA-1,dendritic cell,CL:0000451,Dendritic,Zheng,0.95083535,4.831135,2.215677,-0.061457038,0.77962065,-2.8760715,...,-1.629719,0.9454739,PBMC,Homo sapiens,10x,8.502187,-43.724453,11,9,dendritic cell
10-TTATGAGACGTGAT-1,dendritic cell,CL:0000451,Dendritic,Zheng,-0.09587504,5.612837,2.368608,-1.6328723,-0.0052440166,-2.2625978,...,0.35345224,1.5755694,PBMC,Homo sapiens,10x,10.183571,-41.25579,11,9,dendritic cell
10-TTCCCACTGACACT-1,dendritic cell,CL:0000451,Dendritic,Zheng,0.22103457,5.1280894,2.6506667,-0.3638859,0.18665218,-1.9519036,...,-0.54031163,2.5619676,PBMC,Homo sapiens,10x,10.415549,-41.043446,11,9,dendritic cell


In [46]:
adata.write_h5ad(f'../{dataset}.h5ad')

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cell_ontology_id' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'dataset_name' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Remov

In [47]:
!rm {dataset_download_path}